In [ ]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [ ]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import requests
import json
import os
import errno
from multiprocessing import Pool
import ssl
import time
import urllib
from urllib.request import urlopen
from tqdm import tqdm
import shutil
from pathlib import Path

In [ ]:
dataset_path = '/content/drive/MyDrive/MedPix Dataset'

In [ ]:
links_dict = {
              'Mammo-XCC' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Mammo%20-%20XCC%22%5D&allen=true&allt=true&alli=true',
              'AP': "https://medpix.nlm.nih.gov/search?pln=%5B%22AP%22%5D&allen=true&allt=true&alli=true",
           'Axical' : "https://medpix.nlm.nih.gov/search?pln=%5B%22Axial%22%5D&allen=true&allt=true&alli=true",
             'Coronal' : "https://medpix.nlm.nih.gov/search?pln=%5B%22Coronal%22%5D&allen=true&allt=true&alli=true",
             'Decubitus' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Decubitus%22%5D&allen=true&allt=true&alli=true',
             'Frontal' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Frontal%22%5D&allen=true&allt=true&alli=true',
              'Histology' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Histology%22%5D&allen=true&allt=true&alli=true',
              'Image Plane' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Image%20Plane%22%5D&allen=true&allt=true&alli=true',
              'Lateral' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Lateral%22%5D&allen=true&allt=true&alli=true',
             'Longitudinal' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Longitudinal%22%5D&allen=true&allt=true&alli=true',
             'Mammo-CC' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Mammo%20-%20CC%22%5D&allen=true&allt=true&alli=true',
             'Mammo-MagCC' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Mammo%20-%20Mag%20CC%22%5D&allen=true&allt=true&alli=true',
             'Mammo-MLO' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Mammo%20-%20MLO%22%5D&allen=true&allt=true&alli=true',
             'Multiple or Montage' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Multiple%20or%20Montage%22%5D&allen=true&allt=true&alli=true',
             'NOS' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22NOS%20-%20Not%20specified%22%5D&allen=true&allt=true&alli=true',
             'Oblique' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Oblique%22%5D&allen=true&allt=true&alli=true',
              'Other view' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Other%20View%20(see%20caption)%22%5D&allen=true&allt=true&alli=true',
             'PA' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22PA%22%5D&allen=true&allt=true&alli=true',
               'Sagittal' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Sagittal%22%5D&allen=true&allt=true&alli=true',
              'Transverse' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Transverse%22%5D&allen=true&allt=true&alli=true',
              '3D Reconstruction' : 'https://medpix.nlm.nih.gov/search?pln=%5B%223D%20Reconstruction%22%5D&allen=true&allt=true&alli=true'
}

In [ ]:
links_dict = {
            'Axical' : "https://medpix.nlm.nih.gov/search?pln=%5B%22Axial%22%5D&allen=true&allt=true&alli=true",
            'AP': "https://medpix.nlm.nih.gov/search?pln=%5B%22AP%22%5D&allen=true&allt=true&alli=true",

            'Coronal' : "https://medpix.nlm.nih.gov/search?pln=%5B%22Coronal%22%5D&allen=true&allt=true&alli=true",
             'Frontal' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Frontal%22%5D&allen=true&allt=true&alli=true',
             'Longitudinal' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Longitudinal%22%5D&allen=true&allt=true&alli=true',
            'NOS' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22NOS%20-%20Not%20specified%22%5D&allen=true&allt=true&alli=true',
            'Sagittal' : 'https://medpix.nlm.nih.gov/search?pln=%5B%22Sagittal%22%5D&allen=true&allt=true&alli=true',
}

In [ ]:
def save_to_txt_file(new_data, path):
  myfile = Path(path +'/image_links.txt')
  myfile.touch(exist_ok=True)
  with open(f'{path}/image_links.txt', 'a', encoding='utf-8') as f:
    f.write('\n'.join(new_data))
    f.write('\n')

In [ ]:
def analysis_of_results(driver, past_results_count):
    search_results = driver.find_element(by=By.ID, value='search-results-container').find_element(by=By.CLASS_NAME, value="search-result-list")
    results = [element for element in search_results.find_elements(by=By.CLASS_NAME, value='cow-container')]
    results_count = len(results)
    print(f'past: {past_results_count}, new: {results_count}')

    return (True, results_count) if past_results_count != results_count else (False, past_results_count)

In [ ]:
def new_crawl(driver, last_index, path):
    data = []
    wait = WebDriverWait(driver, 10)
    search_results = driver.find_element(by=By.ID, value='search-results-container').find_element(by=By.CLASS_NAME, value="search-result-list")
    results = [element for element in search_results.find_elements(by=By.CLASS_NAME, value='cow-container')]
    for idx in tqdm(range(last_index, len(results))):
        result = results[idx]
        result.click()
        link =  wait.until(EC.visibility_of_element_located((By.LINK_TEXT, 'View')))

        result_link = link.get_attribute('href')
        data.append(result_link)

    save_to_txt_file(data, path)

In [ ]:
for modality in links_dict:
    link = links_dict[modality]
    current_path = f"{dataset_path}/{modality}"
    os.makedirs(current_path, exist_ok=True)

    driver = webdriver.Chrome('chromedriver',options=chrome_options)
    driver.get(link)
    time.sleep(2)  # Allow 2 seconds for the web page to open

    scroll_pause_time = 2 # You can set your own pause time.
    screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
    i = 1
    search_results = driver.find_element(by=By.ID, value='search-results-container').find_element(by=By.CLASS_NAME, value="search-result-list")
    results = [element for element in search_results.find_elements(by=By.CLASS_NAME, value='cow-container')]
    results_count = len(results)
    total_search = driver.find_element(by=By.CLASS_NAME, value='total-search-results')
    search_count = int(total_search.text.split()[1])
    new_crawl(driver, 0, current_path)

    while True:
        if results_count >= search_count :
            break
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, i * {screen_height});".format(screen_height=screen_height, i=i))
        i += 1
        time.sleep(scroll_pause_time)
        change , count = analysis_of_results(driver, results_count)
        if change :
            new_crawl(driver, results_count, current_path)
            results_count = count

        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * i > scroll_height:
            break

In [ ]:
def save_to_json_file(current_path,new_data):
  myfile = Path(current_path +'/dataset.json')
  myfile.touch(exist_ok=True)
  with open(f'{current_path}/dataset.json', 'r', encoding='utf-8') as f:
    try:
        data = json.load(f)
    except ValueError:
         data = []
  with open(f'{current_path}/dataset.json', 'w', encoding='utf-8') as f:
    data = data + new_data
    json.dump(data, f, ensure_ascii=False, indent=4)

In [ ]:
import json
current_path = f"{dataset_path}/Coronal"
with open(f'{current_path}/image_links.txt', 'r', encoding='utf-8') as f:
        links = f.readlines()
        print(len(links))
        links = list(set(links))
        print(len(links))

4249
2824
